<a href="https://colab.research.google.com/github/alirezakavianifar/gitTutorial/blob/developer/nsga2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here are the parameters that need to be included according to the document:

### Parameters:
1. **ri**: Delivery demand of delivery nodes i.
2. **Pi**: Pickup demand of delivery nodes i.
3. **Sit**: Time when truck tr provides service to delivery nodes i.
4. **Sik**: Time when RPA of truck tr reaches node i (in minutes).
5. **Qk**: Loading capacity of each RPA k.
6. **Qtr**: Weight (load) of truck tr after visiting node i.
7. **aik**: Earliest time RPA is allowed to provide service to distributor i in the hard time window.
8. **bik**: Latest time RPA is allowed to provide service to distributor i in the hard time window.
9. **M**: Optional large number.
10. **ESik**: Earliest time RPA is allowed to provide service to distributor i in the soft time window.
11. **LSik**: Latest time RPA is allowed for RPA to provide service to distributor i in the soft time window.
12. **aiTR**: Earliest time truck is allowed to provide service to distributor i in the hard time window.
13. **biTR**: Latest time truck is allowed to provide service to distributor i in the hard time window.
14. **ESiTR**: Earliest time truck is allowed to provide service to distributor i in the soft time window.
15. **LSiTR**: Latest time truck is allowed to provide service to distributor i in the soft time window.
16. **Ljk**: Weight of the load remaining on the RPA after service to demand point j.
17. **α**: Scaling factor for earliest start time deviation.
18. **β**: Scaling factor for latest start time deviation.
19. **FCk**: Time to end the route of RPA k.
20. **W2**: Cost per unit time deviation from the earliest time allowed in the soft time window.
21. **W3**: Cost per unit time deviation from the latest time allowed in the soft time window.
22. **fiyk**: Fixed cost of using RPA k.
23. **C**: Cost of one charging unit.
24. **AT**: Minimum amount of charging allowed inside the RPA k.
25. **Cpf**: Preparing cost of a unit in pickup nodes i.
26. **fixd**: Cost of constructing distributor candidate location d.
27. **DAY**: Length of a working day.
28. **cpji**: RPA battery consumption from node i to node j.
29. **dij**: Distance between node i and node j.
30. **full**: Battery charging capacity.
31. **pu**: Probability of occurrence of scenarios u.
32. **Vku**: Velocity of RPA k in event u.
33. **capd**: Capacity of candidate location of distributor d.
34. **capf**: Capacity of pickup nodes f.
35. **L0k**: Load on RPA k when leaving the distributor.
36. **Sijk**: Risk of route deriving from node i to node j with RPA k.
37. **siu**: Time to start providing service to demand point i in scenario u.
38. **Eiu**: Time deviation from the earliest time allowed to provide service to demand point i in the soft time window in scenario u.
39. **Liu**: Time deviation from the latest time allowed to provide service to demand point i in the soft time window in scenario u.
40. **po**: Specifies the position in truck tr’s route of node i.
41. **Ai**: Amount of battery available on the RPA k.
42. **cijk**: Energy consumption for an RPA k to fly from location i to location j.
43. **Ck**: Amortized cost of RPA k.
44. **Cbatk**: Amortized cost of the battery of RPA k.
45. **VK**: RPA speed.
46. **CE**: Cost per unit of energy.
47. **CL**: Wage of a RPA operator.
48. **nk**: Number of RPAs of type k an RPA operator can operate simultaneously.
49. **CkM**: Maintenance cost of an RPA of type k.
50. **tkbat**: Time required to replace the battery of RPA type k.
51. **lik**: Latest permissible time for an RPA to provide service at location i.

Let's add these parameters to the code:

```python
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Define the number of objectives
NUM_OBJECTIVES = 4

# Define sets
D = np.arange(5)  # Distributor candidate locations
N0 = np.arange(10)  # Pickup nodes
ϕk = np.arange(5)  # UAV service nodes
ϕ = np.arange(20)  # Service nodes
N = np.arange(25)  # All nodes including depots
N_plus = np.arange(25)  # All nodes a vehicle can visit
TP = [(i, j, k) for i in N0 for j in N_plus for k in N_plus]  # 3-node sorties of UAV
F = np.arange(10)  # Pickup nodes
P = np.arange(10, 20)  # Delivery nodes
K = np.arange(3)  # RPAs
TR = np.arange(2)  # Trucks
U = np.arange(3)  # Different scenarios
P1 = np.arange(5)  # Delivery nodes with recharging
P2 = np.arange(5, 10)  # Delivery nodes without recharging

# Define parameters with random values
ri = np.random.randint(1, 10, size=len(P))  # Delivery demand of delivery nodes
Pi = np.random.randint(1, 10, size=len(P))  # Pickup demand of delivery nodes
Sit = np.random.randint(1, 1440, size=(len(TR), len(P)))  # Service time by trucks (minutes)
Sik = np.random.randint(1, 1440, size=(len(K), len(N0) + len(N_plus)))  # Service time by RPAs (minutes)
Qk = np.random.randint(1, 50, size=len(K))  # Loading capacity of each RPA
Qtr = np.random.randint(1, 100, size=len(TR))  # Weight (load) of truck after visiting nodes (kg)
aik = np.random.randint(1, 1440, size=(len(K), len(D)))  # Earliest time RPA can provide service
bik = np.random.randint(1440, 2880, size=(len(K), len(D)))  # Latest time RPA can provide service
aiTR = np.random.randint(1, 1440, size=(len(TR), len(D)))  # Earliest time truck can provide service
biTR = np.random.randint(1440, 2880, size=(len(TR), len(D)))  # Latest time truck can provide service
dij = np.random.randint(1, 100, size=(len(N), len(N)))  # Distance between nodes
ci_j_k = np.random.randint(1, 50, size=(len(N0), len(N_plus), len(K)))  # Energy consumption for RPA to fly
Vk = np.random.uniform(10, 50, size=len(K))  # RPA speed
ci_j_tr = np.random.randint(1, 50, size=(len(N0), len(N_plus), len(TR)))  # Cost for truck to travel
SLK = np.random.randint(1, 60, size=len(K))  # Time needed to load RPA before launch (minutes)
SRK = np.random.randint(1, 60, size=len(K))  # Time needed to recover RPA upon rendezvous (minutes)
VTR = np.random.uniform(10, 50, size(len(TR)))  # Truck speed (miles per hour)
MiTR = np.random.randint(1, 100, size=(len(TR), len(N0)))  # Truck load capacity with RPA location (kg)
M = 1000  # Optional large number
ESik = np.random.randint(1, 1440, size=(len(K), len(D)))  # Earliest time RPA in soft time window
LSik = np.random.randint(1440, 2880, size=(len(K), len(D)))  # Latest time RPA in soft time window
ESiTR = np.random.randint(1, 1440, size=(len(TR), len(D)))  # Earliest time truck in soft time window
LSiTR = np.random.randint(1440, 2880, size=(len(TR), len(D)))  # Latest time truck in soft time window
Ljk = np.random.randint(1, 100, size=len(N))  # Load remaining on the RPA after service
α = 0.5  # Scaling factor for earliest start time deviation
β = 0.5  # Scaling factor for latest start time deviation
FCk = np.random.randint(1, 50, size=len(K))  # Time to end the route of RPA
W2 = 1  # Cost per unit time deviation from earliest time allowed
W3 = 1  # Cost per unit time deviation from latest time allowed
fiyk = np.random.randint(1, 100, size=len(K))  # Fixed cost of using RPA


C = 10  # Cost of one charging unit
AT = 5  # Minimum amount of charging allowed
Cpf = np.random.randint(1, 50, size(len(F)))  # Preparing cost of a unit in pickup nodes
fixd = np.random.randint(1, 100, size=len(D))  # Cost of constructing distributor location
DAY = 1440  # Length of a working day in minutes
cpji = np.random.randint(1, 50, size=(len(N), len(N)))  # RPA battery consumption
full = 100  # Battery charging capacity
pu = np.random.uniform(0, 1, size=len(U))  # Probability of occurrence of scenarios
Vku = np.random.uniform(10, 50, size=(len(K), len(U)))  # Velocity of RPA in event
capd = np.random.randint(1, 100, size(len(D)))  # Capacity of distributor location
capf = np.random.randint(1, 100, size(len(F)))  # Capacity of pickup nodes
L0k = np.random.randint(1, 50, size(len(K)))  # Load on RPA when leaving the distributor
Sijk = np.random.randint(1, 100, size(len(N), len(N), len(K)))  # Risk of route for RPA
siu = np.random.randint(1, 1440, size(len(P), len(U)))  # Time to start providing service in scenario
Eiu = np.random.randint(1, 1440, size(len(P), len(U)))  # Earliest time deviation in scenario
Liu = np.random.randint(1, 1440, size(len(P), len(U)))  # Latest time deviation in scenario
po = np.random.randint(1, 10, size(len(P)))  # Position in truck's route
Ai = np.random.randint(1, 50, size(len(K)))  # Battery available on the RPA
cijk = np.random.randint(1, 50, size(len(N), len(N), len(K)))  # Energy consumption for RPA
Ck = np.random.randint(1, 100, size(len(K)))  # Amortized cost of RPA
Cbatk = np.random.randint(1, 100, size(len(K)))  # Amortized cost of the battery of RPA
VK = np.random.uniform(10, 50, size(len(K)))  # RPA speed
CE = 0.5  # Cost per unit of energy
CL = 10  # Wage of an RPA operator
nk = np.random.randint(1, 5, size(len(K)))  # Number of RPAs an operator can operate
CkM = np.random.randint(1, 100, size(len(K)))  # Maintenance cost of RPA
tkbat = np.random.randint(1, 60, size(len(K)))  # Time to replace the battery of RPA
lik = np.random.randint(1, 1440, size(len(N)))  # Latest permissible time for RPA to provide service

# Define decision variables as binary for demonstration
def create_individual():
    # Binary decision variables for routes and assignments
    Xij_tr = np.random.randint(0, 2, size=(len(TR), len(N0), len(N_plus)))
    YijB_tr = np.random.randint(0, 2, size(len(TR), len(N0), len(N_plus)))
    RPijB_k = np.random.randint(0, 2, size(len(K), len(N0), len(N_plus)))
    return np.hstack((Xij_tr.flatten(), YijB_tr.flatten(), RPijB_k.flatten()))

# Define objective functions (simplified for demonstration)
def objective_function_1(individual):
    return sum(individual),

def objective_function_2(individual):
    return sum(individual),

def objective_function_3(individual):
    return sum(individual),

def objective_function_4(individual):
    return sum(individual),

# Define NSGA-II using DEAP
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", lambda ind: (objective_function_1(ind), objective_function_2(ind), objective_function_3(ind), objective_function_4(ind)))
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selNSGA2)

# Create population
population = toolbox.population(n=100)

# Apply NSGA-II
algorithms.eaMuPlusLambda(population, toolbox, mu=100, lambda_=200, cxpb=0.7, mutpb=0.3, ngen=50, stats=None, halloffame=None, verbose=True)
```

The code now includes all the parameters as described in the document. Each parameter has been initialized with random values for demonstration purposes. The decision variables and the NSGA-II algorithm setup remain the same. Adjustments might be needed based on specific requirements or constraints in the problem formulation.

In [1]:
import numpy as np

# Sets
n = 10  # Example size, replace with actual size

D = np.array([f'd{i}' for i in range(n)])  # Set of distributor candidate locations
N0 = np.array([i for i in range(2 * n)])  # Set of pickup nodes
phi_k = np.array([f'phi_k_{i}' for i in range(n)])  # Subset of service nodes that the UAV may service
phi = np.array([i for i in range(1, 2 * n + 1)])  # Set of service nodes
N = np.array([i for i in range(2 * n + 2)])  # Set of nodes in the network (including depots)
N_plus = np.array([i for i in range(1, 2 * n + 2)])  # Set of all nodes a vehicle can visit
TP = np.array([(i, j, k) for i in range(n) for j in range(n) for k in range(n)])  # Set of tuples for 3-node sorties of UAV
F = np.array([i for i in range(1, n + 1)])  # Set of all pickup nodes
P = np.array([i for i in range(n + 1, 2 * n + 1)])  # Set of all delivery nodes
K = np.array([f'k{i}' for i in range(n)])  # Set of RPAs
TR = np.array([f'tr{i}' for i in range(n)])  # Set of Trucks
P1 = np.array([i for i in range(n + 1, n + 5)])  # Set of delivery nodes where recharging is possible
P2 = np.array([i for i in range(n + 5, 2 * n + 1)])  # Set of delivery nodes where recharging is not possible
U = np.array([f'u{i}' for i in range(n)])  # Set of different scenarios

# Parameters
ri = np.random.randint(1, 100, size=(2 * n,))  # The amount of delivery demand of delivery nodes
Pi = np.random.randint(1, 100, size=(2 * n,))  # The amount of pickup demand of delivery nodes
Si_tr = np.random.randint(1, 100, size=(n, 2 * n))  # Time when truck tr provides service to delivery nodes
Si_k = np.random.randint(1, 100, size=(n, 2 * n))  # Time when RPA of truck tr reaches node
Qk = np.random.randint(1, 100, size=(n,))  # Loading capacity of each RPA
Qtr = np.random.randint(1, 100, size=(n, 2 * n))  # Weight (load) of truck after visiting node
ai_k = np.random.randint(1, 100, size=(n,))  # The earliest time RPA allowed to provide service to distributor
bi_k = np.random.randint(1, 100, size=(n,))  # The latest time RPA allowed to provide service to distributor
M = 1000  # Optional large number
ESi_k = np.random.randint(1, 100, size=(n,))  # The earliest time RPA allowed to provide service to distributor in soft time window
LSi_k = np.random.randint(1, 100, size=(n,))  # The latest time RPA allowed for RPA to provide service to distributor in soft time window
ai_TR = np.random.randint(1, 100, size=(n,))  # The earliest time truck allowed to provide service to distributor
bi_TR = np.random.randint(1, 100, size=(n,))  # The latest time truck allowed to provide service to distributor
ESi_TR = np.random.randint(1, 100, size=(n,))  # The earliest time truck allowed to provide service to distributor in soft time window
LSi_TR = np.random.randint(1, 100, size=(n,))  # The latest time truck allowed to provide service to distributor in soft time window
Lj_k = np.random.randint(1, 100, size=(n,))  # The weight of the load remaining on the RPA after service to demand point
alpha = 0.5  # Scaling factor for earliest start time deviation
beta = 0.5  # Scaling factor for latest start time deviation
FCk = np.random.randint(1, 100, size=(n,))  # The time to end the route of RPA
ded = np.random.randint(1, 100, size=(n,))  # The center demand of distributor
W2 = 10  # Cost per unit time deviation from the earliest time allowed in the soft time window
W3 = 10  # Cost per unit time deviation from the latest time allowed in the soft time window
fiyk = np.random.randint(1, 100, size=(n,))  # Fixed cost of using RPA
C = 10  # Cost of one charging unit
AT = 5  # Minimum amount of charging allowed inside the RPA
Cpf = np.random.randint(1, 100, size=(n,))  # Preparing cost of a unit in pickup nodes
fixd = np.random.randint(1, 100, size=(n,))  # Cost of constructing distributor candidate location
DAY = 24  # The length of a working day
cpji = np.random.randint(1, 100, size=(2 * n, 2 * n))  # RPA battery consumption from node i to node j
dij = np.random.randint(1, 100, size=(2 * n, 2 * n))  # The distance between node i and node j
full = 100  # Battery charging capacity
pu = np.random.rand(n)  # The probability of occurrence of scenarios
Vku = np.random.randint(1, 100, size=(n, n))  # The velocity of RPA in event u
capd = np.random.randint(1, 100, size=(n,))  # The capacity of candidate location of distributor
capf = np.random.randint(1, 100, size=(n,))  # The capacity of pickup nodes
L0k = np.random.randint(1, 100, size=(n,))  # The load on RPA when leaving the distributor
Sijk = np.random.randint(1, 100, size=(2 * n, 2 * n, n))  # The risk of route deriving from node i to node j with RPA
siu = np.random.randint(1, 100, size=(n, n))  # The time to start providing service to demand point in scenario u
Eiu = np.random.randint(1, 100, size=(n, n))  # The time deviation from the earliest time allowed to provide service to demand point in the soft time window
Liu = np.random.randint(1, 100, size=(n, n))  # The time deviation from the latest time allowed to provide service to demand point in the soft time window
po = np.random.randint(1, 100, size=(n,))  # Specifies the position in truck route of node
Ai = np.random.randint(1, 100, size=(n,))  # The amount of battery available on the RPA
cijk = np.random.randint(1, 100, size=(2 * n, 2 * n, n))  # Energy consumption for a RPA to fly from location i to location j
Ck = np.random.randint(1, 100, size=(n,))  # Amortized cost of RPA
Cbat = np.random.randint(1, 100, size=(n,))  # Amortized cost of the battery of RPA
VK = np.random.randint(1, 100, size=(n,))  # RPA speed
CE = 10  # Cost per unit of energy
CL = 10  # Wage of a RPA operator
nk = np.random.randint(1, 100, size=(n,))  # Number of RPAs of type k a RPA operator can operate simultaneously
CkM = np.random.randint(1, 100, size=(n,))  # Maintenance cost of a RPA
tkbat = np.random.randint(1, 100, size=(n,))  # Time required to replace the battery of RPA
lik = np.random.randint(1, 100, size=(n,))  # The latest permissible time for a RPA to provide service at location i
ljk = np.random.randint(1, 100, size=(n,))  # The latest permissible time to provide service at delivery location j
litr = np.random.randint(1, 100, size=(n,))  # The latest permissible time for a RPA to provide service at location i
ljtr = np.random.randint(1, 100, size=(n,))  # The latest permissible time to provide service at delivery location j
tij = np.random.randint(1, 100, size=(2 * n, 2 * n))  # The travel time between locations
ei = np.random.randint(1, 100, size=(n,))  # Earliest possible pickup time
chko = np.random.randint(1, 100, size=(n,))  # Initial energy in the battery of RPA
chmin = np.random.randint(1, 100, size=(n,))  # Minimum remaining energy required in the battery of RPA
chmax = np.random.randint(1, 100, size=(n,))  # Maximum energy capacity of the battery for RPA
dchj = np.random.randint(1, 100, size=(n,))  # The additional charge gained at charging points
gi = np.random.randint(1, 100, size=(2 * n,))  # Remaining battery energy of a RPA after returning from delivery location
git = np.random.randint(1, 100, size=(2 * n,))  # Auxiliary variable storing the remaining battery energy after returning from delivery location
fik = np.random.randint(1, 100, size=(2 * n,))  # Timing of when a RPA picks-up the package for delivery location

SLK = 10  # Time needed to load RPA before launch
SRK = 10  # Time needed to recover RPA upon rendezvous
VTR = 50  # Truck speed
MiTR = np.random.randint(1, 100, size=(n,))  # Truck load capacity with RPA location
tau_ij = np.random.randint(1, 100, size=(2 * n, 2 * n))  # Time required by truck to move from node i to node j
tau_ijk = np.random.randint(1, 100, size=(2 * n, 2 * n, n))  # Analogous time for RPA
Cij = np.random.randint(1, 100, size=(2 * n, 2 * n))  # Cost for truck to travel from node i to node j
CijB = np.random.randint(1, 100, size=(2 * n, 2 * n))  # Cost to operate RPA between nodes i, j and B
DTR = np.random.randint(1, 100, size=(n,))  # Truck service duration at node
DK = np.random.randint(1, 100, size=(n,))  # RPA service duration at node

# Decision variables (Binary arrays as placeholders)
Xij_tr = np.zeros((n, 2 * n, 2 * n), dtype=int)
YijB_tr = np.zeros((n, 2 * n, 2 * n), dtype=int)
RPijB_k = np.zeros((n, 2 * n, 2 * n), dtype=int)
zij = np.zeros((2 * n, 2 * n), dtype=int)
yi = np.zeros((2 * n,), dtype=int)
xki = np.zeros((n, 2 * n), dtype=int)
XDji = np.zeros((2 * n, 2 * n), dtype=int)
XiDj = np.zeros((2 * n, 2 * n), dtype=int)
Yij = np.zeros((2 * n, 2 * n), dtype=int)



Parameters and decision variables initialized.
